In [2]:
import os
import json
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## Dataset Preview

### COCO Dataset

In [3]:
coco_json = json.load(open('datasets/train.json'))
images = coco_json['images']
annotations = coco_json['annotations']
categories = {k['id']:k['name'] for k in coco_json['categories']}
COLOR_MAP = [
    (171, 222, 230), (203, 170, 203), (255, 255, 181), (255, 204, 182), (243, 176, 195),
    (151, 193, 169), (254, 225, 232), (198, 219, 218), (255, 150, 138), (85, 203, 205)
]

In [4]:
coco_json.keys()

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])

In [5]:
for k in coco_json.keys():
    if k in ["info", "licenses"]:
        print(f"Example {k}: {coco_json[k]}")
    else:
        print(f"Example {k}: {coco_json[k][0]}")

Example info: {'year': 2021, 'version': '1.0', 'description': 'Recycle Trash', 'contributor': 'Upstage', 'url': None, 'date_created': '2021-02-02 01:10:00'}
Example licenses: [{'id': 0, 'name': 'CC BY 4.0', 'url': 'https://creativecommons.org/licenses/by/4.0/deed.ast'}]
Example images: {'width': 1024, 'height': 1024, 'file_name': 'train/0000.jpg', 'license': 0, 'flickr_url': None, 'coco_url': None, 'date_captured': '2020-12-26 14:44:23', 'id': 0}
Example categories: {'id': 0, 'name': 'General trash', 'supercategory': 'General trash'}
Example annotations: {'image_id': 0, 'category_id': 0, 'area': 257301.66, 'bbox': [197.6, 193.7, 547.8, 469.7], 'iscrowd': 0, 'id': 0}


In [6]:
for image in coco_json['images']:
    for k, v in image.items():
        print(f"{k}: {v}")
    break

width: 1024
height: 1024
file_name: train/0000.jpg
license: 0
flickr_url: None
coco_url: None
date_captured: 2020-12-26 14:44:23
id: 0


In [7]:
for annot in coco_json["annotations"]:
    for k, v in annot.items():
        print(f"{k}: {v}")
    break

image_id: 0
category_id: 0
area: 257301.66
bbox: [197.6, 193.7, 547.8, 469.7]
iscrowd: 0
id: 0


## Dataset EDA

### Number of Classes

In [89]:
def vis_bar_plot(idx: np.ndarray, count: list, fig_w: int, fig_h: int, text_h: int, title: str):
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    ax.bar(
        idx, count, width=0.7, 
        ec='red', lw=2, color="tomato"
    )

    for i, v in zip(idx, count):
        ax.text(
            i, v+text_h, str(v), ha='center', va='bottom',
            fontweight='bold', fontsize=10
        )

    ax.set_title(title, fontsize=14)
    plt.show()

In [ ]:
idx = np.array([categories[i] for i in range(len(coco_json['categories']))],)
count = [len([annot for annot in coco_json['annotations'] if annot['category_id'] == i]) for i in range(len(coco_json['categories']))]
vis_bar_plot(idx, count, 12, 7, 50, "Object Count per Category")


### Random Visualization

In [22]:
def random_vis(root_dir: str, rows: int, cols: int):
    idxs = random.sample(list(range(len(images))), rows*cols)
    fig, axes = plt.subplots(rows, cols, figsize=(20, 20))

    for i, idx in enumerate(idxs):
        r, c = i//cols, i%cols
        img_path = os.path.join(root_dir, images[idx]['file_name'])
        img = Image.open(img_path).convert("RGB")
        candits = [k for k in annotations if k['image_id'] == images[idx]['id']]
        for candit in candits:
            bbox = candit['bbox']
            bbox = [int(k) for k in bbox]
            cat_id = candit['category_id']
            cat_name = categories[cat_id]
            color = [num/255 for num in COLOR_MAP[cat_id%10]]
            bbox_rect = patches.Rectangle(
                (bbox[0], bbox[1]), 
                bbox[2], bbox[3],       
                linewidth=3,        
                edgecolor=color,        
                facecolor='none'        
            )
            axes[r][c].add_patch(bbox_rect)
            axes[r][c].annotate(
                cat_name, 
                (bbox[0]+3, bbox[1]-10), 
                color='black', 
                weight='bold', 
                fontsize=10,
                backgroundcolor=color
            )
            
        axes[r][c].imshow(img)
        axes[r][c].axis('off')
        axes[r][c].set_title(img_path.split('/')[-1], fontsize=20)
    
    plt.tight_layout()
    plt.show()

In [ ]:
random_vis("datasets", 2, 2)

In [34]:
def random_vis_with_raw_image(root_dir: str, idx: int):
    fig, axes = plt.subplots(1, 2, figsize=(12, 7))
    img_path = os.path.join(root_dir, images[idx]['file_name'])
    img = Image.open(img_path).convert("RGB")
    axes[0].imshow(img)
    axes[0].axis('off')
    
    candits = [k for k in annotations if k['image_id'] == images[idx]['id']]
    for candit in candits:
        bbox = candit['bbox']
        bbox = [int(k) for k in bbox]
        cat_id = candit['category_id']
        cat_name = categories[cat_id]
        color = [num/255 for num in COLOR_MAP[cat_id%10]]
        bbox_rect = patches.Rectangle(
            (bbox[0], bbox[1]), 
            bbox[2], bbox[3],       
            linewidth=3,        
            edgecolor=color,        
            facecolor='none'        
        )
        axes[0].add_patch(bbox_rect)
        axes[0].annotate(
            cat_name, 
            (bbox[0]+3, bbox[1]-10), 
            color='black', 
            weight='bold', 
            fontsize=10,
            backgroundcolor=color
        )
    axes[1].imshow(img)
    axes[1].axis('off') 
   
    plt.tight_layout()
    plt.show()       

In [ ]:
random_vis_with_raw_image("datasets", 3654)

In [ ]:
random_vis_with_raw_image("datasets", 4197)

In [ ]:
random_vis_with_raw_image("datasets", 2932)

In [ ]:
random_vis_with_raw_image("datasets", 4742)

In [ ]:
random_vis_with_raw_image("datasets", 3102)

In [ ]:
random_vis_with_raw_image("datasets", 1071)

### Number of Boxes per image 

In [74]:
from collections import defaultdict, Counter

In [93]:
num_of_bbox_per_image = defaultdict(int)
unique_class_per_image = defaultdict(int)
for image in images:
    candits = [k for k in annotations if k['image_id'] == image['id']]
    num_of_bbox_per_image[len(candits)] += 1
    unique_class = len(set([k['category_id'] for k in candits]))
    unique_class_per_image[unique_class] += 1
        
num_of_bbox_per_image = sorted(num_of_bbox_per_image.items(), key=lambda x: x[0])
unique_class_per_image = sorted(unique_class_per_image.items(), key=lambda x: x[0])

In [ ]:
idx = np.array([k[0] for k in num_of_bbox_per_image])
count = np.array([k[1] for k in num_of_bbox_per_image])
vis_bar_plot(idx, count, 22, 7, 20, "Number of Bounding Boxes per Image")

In [ ]:
idx = np.array([k[0] for k in unique_class_per_image])
count = np.array([k[1] for k in unique_class_per_image])
vis_bar_plot(idx, count, 10, 7, 20, "Number of Unique Classes per Image")

### Distribution of Bounding box area

object별 area가 다르기 때문에 구간별 clipping을 통해 시각화 진행
- Train image의 크기가 1024 * 1024인 점을 고려했을 때, Size가 너무 작은 object는 오히려 학습을 방해할 것이라 생각.

구간을 어떻게 나눌까?
- bar chart보다 histogram이 좋을 것이라 생각함.

In [98]:
bbox_area = [k['area'] for k in annotations]

In [119]:
print(f"Length of list: {len(bbox_area)}")
print(f"Max area: {max(bbox_area)}")
print(f"Min area: {min(bbox_area)}\n")

print(f"Count of under than 1000: {len(list(filter(lambda x: x<1000, bbox_area)))}")
print(f"Count of under than 2000: {len(list(filter(lambda x: x<2000, bbox_area)))}")
print(f"Count of under than 3000: {len(list(filter(lambda x: x<3000, bbox_area)))}\n") 

print(f"Count of over than 300000: {len(list(filter(lambda x: x>300000, bbox_area)))}")
print(f"Count of over than 400000: {len(list(filter(lambda x: x>400000, bbox_area)))}")
print(f"Count of over than 500000: {len(list(filter(lambda x: x>500000, bbox_area)))}") 

Length of list: 23144
Max area: 1048371.21
Min area: 0.56

Count of under than 1000: 297
Count of under than 2000: 1374
Count of under than 3000: 2208

Count of over than 300000: 1703
Count of over than 400000: 918
Count of over than 500000: 508


Area가 2000 (45 x 45) 보다 적고, 400000 (600 x 600)만보다 큰 object는 날려도 되지 않을까?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 7))
ax.hist(bbox_area, bins=500, color="tomato")
ax.set_title("Area Distribution", fontsize=14)
plt.show()